Deeper understanding of the problem
In the previous topic, you have understood the need for decision tree and its comparison to linear models. In this topic, we will understand the bank marketing data in more detail. We will look at some more features of the dataset. These features are a subset of the original 17 features of the original dataset on which some preprocessing is done. Before we apply machine learning algorithm on any data, this analysis is very essential, and some of this has already been covered in the data pre-processing module. Let us look at the features of the entire data.
Input variables:
bank client data:
age (numeric)
balance: average yearly balance, in euros (numeric)
housing_cat: has housing loan? (binary: "yes","no")
related with the last contact of the current campaign:
duration: last contact duration, in seconds (numeric)
other attributes:
campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, 10000 means client was not previously contacted)
poutcome_success: whether the previous marketing campaign was successful(categorical: 1("success"),0("failure or unknown"))
output variable (desired target):
y - has the client subscribed a term deposit? (binary: "yes","no")


# Q.1)
 
Understanding the data
In this task, you will load the dataset that you will be working on throughout this concept. Understanding the data is the first step before applying any ML algorithm.
Instructions
Load the features data from feature_path using the read_csv() method from pandas. Save it as bank_full
Load the labels from label_path using the read_csv() method. Save it as bank_full_labels
Look at the first 5 rows of the data using the head() method.


In [1]:
# import packages
import pandas as pd
feature_path='data_subset.csv'
label_path='data_subset_label.csv'
# Code starts here
bank_small=pd.read_csv(feature_path)
bank_small_labels=pd.read_csv(label_path)



In [2]:
bank_small

,age,balance,duration,campaign,pdays,housing_cat,poutcome_success
0,59,2343,1042,1,10000,1,0
1,56,45,1467,1,10000,0,0
2,41,1270,1389,1,10000,1,0
3,55,2476,579,1,10000,1,0
4,54,184,673,2,10000,0,0
...,...,...,...,...,...,...,...
11157,33,1,257,1,10000,1,0
11158,39,733,83,4,10000,0,0
11159,32,29,156,2,10000,0,0
11160,43,0,9,2,172,0,0


In [4]:
bank_small_labels

,deposit_cat
0,1
1,1
2,1
3,1
4,1
...,...
11157,0
11158,0
11159,0
11160,0


# Quiz: What is decision tree

QUESTION 1 OF 2
Decision Tree is
Flow-Chart
A tree you grow in your garden that helps you make life decisions
Flow-Chart & Structure in which internal node represents test on an attribute, each branch represents outcome of test and each leaf node represents class label
None of the mentioned

QUESTION 2 OF 2
A _________ is a support tool that uses a tree-like graph or model of decisions and their possible consequences, including chance event outcomes, resource costs, and utility.
Decision tree
Graphs
Trees
Neural Networks


# Entropy calculation for entire set
Now we will look at a hands-on implementation of entropy on the toy dataset in python.
We define a modified log2 code to handle the log zero error. 
Then we calculate entropy on the entire dataset.


In [5]:
from math import log2
def mod_log2(temp): # modified log2 to prevent zero error
    if temp:
        return log2(temp)
    else:
        return log2(temp+0.000000000000000000000000000001) # just a small value to add! 
# entropy of dataset before split
total_label_dist = bank_small_labels['deposit_cat'].value_counts().to_dict()

# total number of points
num_total_data = bank_small_labels.count()[0]

# number of positive class points
pos_class_count = bank_small_labels[bank_small_labels['deposit_cat']==1].count()[0] 
# number of negative class points
neg_class_count = bank_small_labels[bank_small_labels['deposit_cat']==0].count()[0] 

# calculate the full entropy of the data
entropy_full_data = -(pos_class_count/num_total_data)*mod_log2(pos_class_count/num_total_data) + \
-(neg_class_count/num_total_data)*mod_log2(neg_class_count/num_total_data)
print("The entropy of the data before splitting is %.2f.\n" % entropy_full_data)


The entropy of the data before splitting is 1.00.



# Now let's look at the implementation of the entropy after the split on our toy data. 
We first split the data based on the splitting point and then get the distribution of class labels
Then we take counts of data in the positive class (with class label 1) and negative class.


In [6]:
# take the logical array of the indices for the data split
age_left_split = bank_small['age'] < 39

# taking the logical not operator to get the indices for the right split
age_right_split = ~age_left_split

# getting the class labels of the left split
age_left_split_labels = bank_small_labels[age_left_split]

# number of elements in the left split
num_left_split = age_left_split_labels.count()[0]

# getting the class labels of the right split
age_right_split_labels = bank_small_labels[age_right_split]

# number of elements in the right split
num_right_split = age_right_split_labels.count()[0] 

# get the class label distribution 
left_split_dist = age_left_split_labels['deposit_cat'].value_counts().to_dict()
right_split_dist = age_right_split_labels['deposit_cat'].value_counts().to_dict()



For each of the splits, we individually calculate the entropy and then calculate the total entropy after the split.


In [7]:
# entropy of dataset after split - left side       
try:
    pos_cc_left_split = left_split_dist[1] # positive class counts
except KeyError:
    pos_cc_left_split = 0

try:
    neg_cc_left_split = left_split_dist[0] # negative class counts
except KeyError:
    neg_cc_left_split = 0


entropy_age_left_split = -1*((pos_cc_left_split/num_left_split)*mod_log2(pos_cc_left_split/num_left_split)) + \
-1*((neg_cc_left_split/num_left_split)*mod_log2(neg_cc_left_split/num_left_split))

# entropy of dataset after split - right side
try:
    pos_cc_right_split = right_split_dist[1] # positive class counts
except KeyError:
    pos_cc_right_split = 0

try:
    neg_cc_right_split = right_split_dist[0] # negative class counts
except KeyError:
    neg_cc_right_split = 0

entropy_age_right_split = -1*((pos_cc_right_split/num_right_split)*mod_log2(pos_cc_right_split/num_right_split)) + \
-1*((neg_cc_right_split/num_right_split)*mod_log2(neg_cc_right_split/num_right_split))

# combining both to get the correct entropy
entropy_after_split_age = (num_left_split/num_total_data)*entropy_age_left_split + \
(num_right_split/num_total_data)*entropy_age_right_split
print("The entropy of the data after splitting with age is %.2f.\n" % entropy_after_split_age)   


The entropy of the data after splitting with age is 1.00.



In a similar way, we calculate the entropy of data after splitting at balance<1500.
After that, we calculate the information gain for each attribute.


In [8]:
#total_entropy - entropy_after_split_age
print("The information gain after splitting with age = 39 is %.2f.\n" % (entropy_full_data - entropy_after_split_age))
print("The information gain after splitting with balance is %.2f.\n" % (entropy_full_data - entropy_after_split_bal))



The information gain after splitting with age = 39 is 0.00.



NameError: name 'entropy_after_split_bal' is not defined

# GINI

In [9]:
try:
    pos_cc_left_split = left_split_dist[1] # positive class counts
except KeyError:
    pos_cc_left_split = 0

try:
    neg_cc_left_split = left_split_dist[0] # negative class counts
except KeyError:
    neg_cc_left_split = 0

gini_age_left_split = 1 - ((pos_cc_left_split/num_left_split)**2 + (neg_cc_left_split/num_left_split)**2)

try:
    pos_cc_right_split = right_split_dist[1] # positive class counts
except KeyError:
    pos_cc_right_split = 0

try:
    neg_cc_right_split = right_split_dist[0] # negative class counts
except KeyError:
    neg_cc_right_split = 0

gini_age_right_split = 1 - ((pos_cc_right_split/num_right_split)**2 + (neg_cc_right_split/num_right_split)**2)

gini_age_39 = (num_left_split/num_total_data)*gini_age_left_split + \
(num_right_split/num_total_data)*gini_age_right_split

print("The gini value of data after splitting with age=39 is %.2f.\n" % gini_age_39)  


The gini value of data after splitting with age=39 is 0.50.



# Using nominal values for splitting the decision tree


In [10]:
# getting the split of the logical array
housing_split_yes = bank_small['housing_cat']==1
housing_split_no = ~housing_split_yes

# get the class labels for the data split on housing_cat
housing_split_yes_labels = bank_small_labels[housing_split_yes]
housing_split_no_labels = bank_small_labels[housing_split_no]


# Quiz: How to build a decision tree

QUESTION 1 OF 1
In Decision Tree, by comparing the impurity across all possible splits in all possible Predictors, the next split is choosen. Impurity in Decision Tree is generally measured using what?

RMSE
Entropy, Ginni-Index
ROC
MAPE


ans:Entropy, Ginni-Index

In [12]:
# import packages
import pandas as pd
feature_path='data_subset.csv'
label_path='data_subset_label.csv'
# Code starts here
bank_full=pd.read_csv(feature_path)
bank_full_labels=pd.read_csv(label_path)

In [13]:
bank_full

,age,balance,duration,campaign,pdays,housing_cat,poutcome_success
0,59,2343,1042,1,10000,1,0
1,56,45,1467,1,10000,0,0
2,41,1270,1389,1,10000,1,0
3,55,2476,579,1,10000,1,0
4,54,184,673,2,10000,0,0
...,...,...,...,...,...,...,...
11157,33,1,257,1,10000,1,0
11158,39,733,83,4,10000,0,0
11159,32,29,156,2,10000,0,0
11160,43,0,9,2,172,0,0


In [14]:
bank_full_labels

,deposit_cat
0,1
1,1
2,1
3,1
4,1
...,...
11157,0
11158,0
11159,0
11160,0


# Q.2)Learning a decision tree on given data

In this task, you will generate a decision tree on the data and also visualize it.
Intructions
Use the train_test_split() module to split the data and labels into train and test sets. Use 80% of the data for training and 20% of the data for testing. Save them as data_train(training features), data_test (test features), label_train (training labels) and label_test (test labels). Use random_state = 6.
Use the tree.DecisionTreeClassifier() to initialize a decision tree object as dt2. Pass the parameter max_depth=2 while initializing the object. (You can also pass the parameter criterion=entropy/criterion=gini. It is criterion=gini by default).
Use the fit() method of the decision tree object to train the decision tree on the training data i.e. on data_train and label_train
Use the tree.export_graphviz() module to create the graphic visualization object of the learned tree 'dt2' .Input the parameters dt2, out_file=None, feature_names=bank_full.columns, filled = True, class_names=['term_deposit_yes','term_deposit_no'] and save it as dot_data
Use the pydotplus.graph_from_dot_data() module to draw the graph. Save it as graph_big
To display the graph we save the above created image into a file and display it using matplotlib. We have given you the code for the same. Do not modify the code for Show Graph given to you


In [29]:
# import packages
 
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics

import pydotplus
from IPython.display import Image

 


In [30]:
data_train,data_test,label_train,label_test=train_test_split(bank_full,bank_full_labels,test_size=0.2,random_state = 6)

dt2=tree.DecisionTreeClassifier(max_depth=2,criterion='gini')#criterion=entropy/criterion=gini its gini by default

dt2.fit(data_train,label_train)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [31]:
dot_data = tree.export_graphviz(dt2, out_file=None, feature_names=bank_full.columns, filled = True, 
                                class_names=['term_deposit_yes','term_deposit_no']) 

In [35]:
# Draw graph
graph_big = pydotplus.graph_from_dot_data(dot_data) 
 
 
# show graph - do not delete/modify the code below this line
img_path = 'C:\UsersA\dmin\Desktop\1.png'
graph_big.write_png(img_path)
 
plt.figure(figsize=(14,18))
plt.imshow(plt.imread(img_path))
plt.axis('off')
plt.show()
# Code ends here


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-35-02cd73162dbb>, line 6)

# Q.3)Getting the accuracy of test data using decision tree

We have earlier looked at the accuracy of test data on toy data. Let us look at accuracy on real test data
Instructions
Initialize a new decision tree 'dt2' with max_depth=3
Fit 'dt2' model on 'data_train' and label_train
Use 'the score() method of dt2 to find the accuracy of the learned decision tree on the training data and uses arguments data_train and label_train. (To learn how well model has learnt on training data) Save it as dt2_score_train
Use the score() method this time to find the accuracy of the learned decision tree on the test data i.e. data_test and label_test. Save it as dt2_score_test
Print the train and test accuracy scores


In [40]:
# Code starts here
from sklearn.tree import DecisionTreeClassifier
dt2 = DecisionTreeClassifier(max_depth=3)
dt2.fit(data_train,label_train)
dt2_score_train = dt2.score(data_train,label_train)
dt2_score_test =  dt2.score(data_test,label_test)

 


In [47]:
print("Socre with depth 3:",dt2_score_train)

Socre with depth 3: 0.7550677567476761


In [48]:
dt2_score_test

0.7586206896551724

# Q.4)Observe overfitting of decision tree on data

In this task, we will try to see if there is overfitting on the training data, as we increase the depth of decision tree on the complete data.

Instructions
Initialize a new decision tree classifier dt4 with criterion=entropy , max_depth=4 and random_state=6
Fit dt4 on data_train and label_train
Find accuracy on training data as dt4_score_train using .score() method of dt4
Find accuracy on test data as dt4_score_test using .score() method of dt4 on test data this time

Now initalize a new decision tree classifier dt_full with arguments as criterion=entropy and random_state=6
Fit dt_full on data_train and label_train
Find accuracy on training data as dt_full_score_train using .score() method of dt_full
Find accuracy on test data as dt_full_score_test using .score() method of dt_full on test data this time


In [43]:
from sklearn.tree import DecisionTreeClassifier
dt4=DecisionTreeClassifier(criterion='entropy' , max_depth=4 ,random_state=6)
dt4.fit(data_train,label_train)
dt4_score_train = dt4.score(data_train,label_train)
dt4_score_test = dt4.score(data_test,label_test)
 


In [49]:
print("Socre with depth 4:",dt4_score_train)

Socre with depth 4: 0.7842983536790235


In [45]:
dt4_score_test

0.7984773846842812

In [51]:
dt_full = DecisionTreeClassifier(criterion='entropy' ,random_state=6)
dt_full.fit(data_train,label_train)
dt_full_score_train = dt_full.score(data_train,label_train)
dt_full_score_test = dt_full.score(data_test,label_test)


In [52]:
dt_full_score_train # wen depth increases frm 4 to 5 it becomes overfitting

1.0

In [53]:
dt_full_score_test

0.7344379758172862

In [56]:
# import packages
 
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics

import pydotplus
from IPython.display import Image


In [57]:
# Create decision tree classifer object
dt10 = tree.DecisionTreeClassifier(criterion='entropy', min_samples_leaf=3)

# Train the model
dt10.fit(bank_small, bank_small_labels)

# Create DOT data for visualizing the tree
dot_data = tree.export_graphviz(dt10, out_file=None,
                                feature_names=bank_small.columns, filled = True,  
                                class_names=['term_deposit_no','term_deposit_yes'])

# Draw graph
graph_small = pydotplus.graph_from_dot_data(dot_data)  

# Show graph
Image(graph_small.create_png())


InvocationException: GraphViz's executables not found

# Q.5)Reduce overfitting by limiting the number of samples of leaf

We have seen how max_depth helps in reducing overfitting. Now let's see how min_samples_leaf helps in reducing overfitting.
Instructions
Create new decision tree dt_msl with criterion='entropy' and min_samples_leaf=100
Use the fit() method of the decision tree object to train the decision tree on the training data i.e. data_train and label_train
Use the score() method of dt_msl to find the accuracy of the learned decision tree on the training data. (To learn how well model has learnt on training data) Save it as dt_msl_score_train
Use the score() method to find the accuracy of the learned decision tree on the test data i.e. data_test and label_test. Save it as dt_msl_score_test
Print the train and test accuracy scores


In [58]:
# Code starts here
from sklearn.tree import DecisionTreeClassifier
 
dt_msl=DecisionTreeClassifier(criterion='entropy' , min_samples_leaf=100)
dt_msl.fit(data_train,label_train)
dt_msl_score_train = dt_msl.score(data_train,label_train)
dt_msl_score_test = dt_msl.score(data_test,label_test)
print(dt_msl_score_train,dt_msl_score_test)
 
# Code ends here


0.8102811065068877 0.812360053739364


# Quiz: Overfitting in decision tree

QUESTION 1 OF 2
How we can avoid the overfitting in Decision Tree?
CHAID(Stopping the tree growth)
Pruning the full grown tree
Both the above methods
None of the above

QUESTION  2 OF 2
Pre-pruning the decision tree may result in _________.
Overfitting
Underfitting
